In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial as ssp
import cv2

from voronoi_plot_2d import voronoi_plot_2d

In [2]:
output_size: int = 1024
draw_size: int = output_size + 40
generation_number: int = 1000

In [3]:
def generate_roads(filename: str):
    # randomize number of points and their location
    number_of_points: int = np.random.randint(4, 15)
    points = np.array(
        [
            [np.random.randint(0, draw_size), np.random.randint(0, draw_size)]
            for i in range(number_of_points)
        ]
    )
    points = points / draw_size

    # calculate voronoi with provided points
    vor = ssp.Voronoi(points, furthest_site=True, incremental=True)

    fig = plt.figure(facecolor="black", frameon=False, clear=True)
    fig.set_size_inches(output_size / fig.dpi, output_size / fig.dpi)

    axes = fig.add_subplot(111)
    axes.axis("off")

    # draw voronoi diagram
    voronoi_plot_2d(
        vor=vor,
        ax=axes,
        show_points=False,
        show_vertices=False,
        line_colors="black",
        line_styles="solid",
        line_widths=[10, 10, 10, 6, 6, 6, 6, 6, 3],
        line_alpha=1,
    )

    # save original white background, black lines image
    plt.tight_layout(pad=0)
    plt.savefig(filename, transparent=True, facecolor="black")
    plt.close()

    # load to cv2 (believe me, it's easier this way than to convert directly - thanks matplotlib)
    img = cv2.imread(filename=filename, flags=cv2.IMREAD_GRAYSCALE)

    # perform image binarization (prevents weird artefacts on negation)
    _, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

    # make image negative (matplotlib cannot handle black backgrounds correctly for some reason)
    img = 255 - img

    # smooth-out rough edges of roads
    img = cv2.medianBlur(src=img, ksize=5)
    img = cv2.GaussianBlur(src=img, ksize=(3, 3), sigmaX=0)

    # save negative to file
    cv2.imwrite(filename=filename, img=img)

In [5]:
for i in range(generation_number):
    generate_roads(f"out/road{i}.jpg")
